In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
from scipy.spatial.transform import Rotation

# load data

In [ ]:
folder_path='sample_data/Room_back_home-2024-12-29_19-50-26/'
folder_path_img = folder_path+'/Camera'

gyro = pd.read_csv(folder_path+'Gyroscope.csv')
gyro['time_diff_s'] = gyro['time'].diff().fillna(0) / 1e9
gyro['roll'] = (gyro['x']*gyro['time_diff_s']).cumsum()
gyro['pitch'] = (gyro['y']*gyro['time_diff_s']).cumsum()
gyro['yaw'] = (gyro['z']*gyro['time_diff_s']).cumsum()
gyro.drop(['x','y','z'],axis=1,inplace=True)

acc = pd.read_csv(folder_path+'/Accelerometer.csv')

motion = pd.merge(gyro, acc, on=['time','seconds_elapsed'],how='inner')
motion.rename(columns={'x': 'ax','y': 'ay','z': 'az'}, inplace=True)


image_filenames = sorted(os.listdir(folder_path_img))[1:-1]
image_timestamps = [float(filename.split('.')[0]) for filename in image_filenames] 

def synchronize_data(imu_data, image_timestamps):
    synchronized_data = []
    for img_time in image_timestamps:
        # Find closest IMU timestamp
        closest_imu_index = (np.abs(imu_data['time'] - img_time*1e6)).argmin()
        synchronized_data.append({
            'image_time': img_time,
            'imu_roll': imu_data.iloc[closest_imu_index]['roll'],
            'imu_pitch': imu_data.iloc[closest_imu_index]['pitch'],
            'imu_yaw': imu_data.iloc[closest_imu_index]['yaw'],
            'imu_ax': imu_data.iloc[closest_imu_index]['ax'],
            'imu_ay': imu_data.iloc[closest_imu_index]['ay'],
            'imu_az': imu_data.iloc[closest_imu_index]['az'],
            'image_path': os.path.join(folder_path_img, image_filenames[image_timestamps.index(img_time)])
        })
    return pd.DataFrame(synchronized_data)

synchronized_synchronized_df = synchronize_data(motion, image_timestamps)
synchronized_synchronized_df['time_diff_s'] = synchronized_synchronized_df['image_time'].diff().fillna(0) / 1e3

In [ ]:
synchronized_synchronized_df.head()

In [ ]:
# Step 4: Pose Estimation from IMU Data
def compute_pose_from_imu(roll, pitch, yaw, ax,ay,az, dt):
    # Create rotation matrix from Euler angles
    R_x = np.array([[1, 0, 0],
                    [0, np.cos(roll), -np.sin(roll)],
                    [0, np.sin(roll), np.cos(roll)]])
    
    R_y = np.array([[np.cos(pitch), 0, np.sin(pitch)],
                    [0, 1, 0],
                    [-np.sin(pitch), 0, np.cos(pitch)]])
    
    R_z = np.array([[np.cos(yaw), -np.sin(yaw), 0],
                    [np.sin(yaw), np.cos(yaw), 0],
                    [0, 0, 1]])

    # Combined rotation matrix
    R = R_z @ R_y @ R_x

    # Integrate acceleration to get velocity
    velocity_change = np.array([ax * dt, ay * dt, az * dt])
    
    # Transform velocity change into global frame using rotation matrix
    global_velocity_change = R @ velocity_change
    
    return R, global_velocity_change
    

In [ ]:
trajectory = []
velocity = np.zeros(3)  # Initialize velocity

for index in range(len(synchronized_synchronized_df)):
    roll = synchronized_synchronized_df.iloc[index]['imu_roll']
    pitch = synchronized_synchronized_df.iloc[index]['imu_pitch']
    yaw = synchronized_synchronized_df.iloc[index]['imu_yaw']

    dt = synchronized_synchronized_df.iloc[index]['time_diff_s']
    
    # Get IMU acceleration data
    ax = synchronized_synchronized_df.iloc[index]['imu_ax']
    ay = synchronized_synchronized_df.iloc[index]['imu_ay']
    az = synchronized_synchronized_df.iloc[index]['imu_az']

    # Compute pose from IMU data
    R, velocity_change = compute_pose_from_imu(roll, pitch, yaw, ax, ay, az, dt)
    
    # Update velocity based on acceleration (integrate)
    velocity += velocity_change
    
    if index == 0:
        current_position = np.zeros(3)  # Initial position at origin
    else:
        current_position += velocity * dt  # Integrate velocity to get position
    
    trajectory.append(current_position.copy())
trajectory = np.array(trajectory)
synchronized_synchronized_df['x']=trajectory[:,0]
synchronized_synchronized_df['y']=trajectory[:,1]
synchronized_synchronized_df['z']=trajectory[:,2]

In [ ]:
synchronized_synchronized_df

# triangulation

In [ ]:
def load_image(path):
    return cv2.imread(path, cv2.IMREAD_GRAYSCALE)

def detect_and_describe(image):
    sift = cv2.SIFT_create()
    keypoints, descriptors = sift.detectAndCompute(image, None)
    return keypoints, descriptors

def match_features(desc1, desc2):
    bf = cv2.BFMatcher()
    matches = bf.knnMatch(desc1, desc2, k=2)
    good = []
    for m, n in matches:
        if m.distance < 0.75 * n.distance:
            good.append(m)
    return good

def linear_triangulation(p1, p2, P1, P2):
    num_points = p1.shape[1]
    A = np.zeros((4*num_points, 4))
    for i in range(num_points):
        x1, y1 = p1[0:2, i]
        x2, y2 = p2[0:2, i]
        A[4*i] = x1*P1[2] - P1[0]
        A[4*i+1] = y1*P1[2] - P1[1]
        A[4*i+2] = x2*P2[2] - P2[0]
        A[4*i+3] = y2*P2[2] - P2[1]
    
    _, _, Vt = np.linalg.svd(A)
    X = Vt[-1, :3] #/ Vt[-1, 3]
    return X

In [ ]:
K = np.array([[730, 0, 620], [0, 730, 360], [0, 0, 1]])  # Example values

all_3d_points = []

for i in range(len(synchronized_df) - 1):
    img1 = load_image(synchronized_df.iloc[i]['image_path'])
    img2 = load_image(synchronized_df.iloc[i+1]['image_path'])
    
    kp1, desc1 = detect_and_describe(img1)
    kp2, desc2 = detect_and_describe(img2)
    
    matches = match_features(desc1, desc2)
    
    R1 = Rotation.from_euler('xyz', [synchronized_df.iloc[i]['imu_roll'], synchronized_df.iloc[i]['imu_pitch'], synchronized_df.iloc[i]['imu_yaw']]).as_matrix()
    t1 = np.array([synchronized_df.iloc[i]['x'], synchronized_df.iloc[i]['y'], synchronized_df.iloc[i]['z']])
    R2 = Rotation.from_euler('xyz', [synchronized_df.iloc[i+1]['imu_roll'], synchronized_df.iloc[i+1]['imu_pitch'], synchronized_df.iloc[i+1]['imu_yaw']]).as_matrix()
    t2 = np.array([synchronized_df.iloc[i+1]['x'], synchronized_df.iloc[i+1]['y'], synchronized_df.iloc[i+1]['z']])
    
    P1 = K @ np.hstack((R1, t1.reshape(3, 1)))
    P2 = K @ np.hstack((R2, t2.reshape(3, 1)))
    
    points1 = np.array([kp1[m.queryIdx].pt for m in matches]).T[:,0:1]
    points2 = np.array([kp2[m.trainIdx].pt for m in matches]).T[:,0:1]
    
    # Normalize the points
    points1_norm = np.linalg.inv(K) @ np.vstack((points1, np.ones(points1.shape[1])))
    points2_norm = np.linalg.inv(K) @ np.vstack((points2, np.ones(points2.shape[1])))
    
    # Triangulate all points simultaneously
    points_3d = linear_triangulation(points1_norm, points2_norm, P1, P2)
    
    all_3d_points.extend(points_3d)
    break

all_3d_points = np.array(all_3d_points)

In [ ]:
points_3d,points1,points2